- Qual marca tem mais comentários positivos?
- Qual Website tem mais comentários positivos?
- Quais são as principais features que são citadas para cada website/marca?
- Quais as principais features que são citadas quando os comentários são negativos/positivos?

- Limpar os dados de texto
  - Pontuação
  - Letra Minuscula
  - Stop Words
- Avaliar Subjetividade e polaridade do comentário para cada linha
  - Usando TextBlob Sentiment
- Agrupar linhas por marcas e por websites
- Pegar a média da polaridade deles e ordenar pra pegar os mais positivos e os mais negativos
- Fazer uma lista de features com base nas general features (keys)
- Verificar quantas vezes essas features aparecem nos comentários
- Agrupar por comentarios positivos e negativos


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.util import bigrams, trigrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob import TextBlob

import re
import string

/home/joaogarcia/Documents/DataScience/venv/lib/python3.6/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [2]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/joaogarcia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/joaogarcia/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
data = pd.read_csv('./Dataset/NLP_Data.csv', index_col=0, dtype={'General Features': object})
data.drop(columns=['URL', 'ID'], inplace=True)
data.head()

,Website,Product,Brand,Description,Technical Features,General Features,Comment#1,Comment#2,Rating,Available Colors,No. Colors
0,lowes,Refrigerators,General Electric,GE 25.1-cu ft Side-by-Side Refrigerator with I...,'25.1':2 'black':13 'by':7 'cu':3 'ft':4 'ge':...,"{'brand': ['GE', 'Ge'], 'model': ['GSS25IBNTS'...",Functional,Pros: fingerprint resistant so you don't have ...,3.0,"['1000859768', '1000859852']",2
1,lowes,Refrigerators,Frigidaire,Frigidaire Gallery 21.7-cu ft Counter-depth Fr...,'21.7':3 'counter':7 'counter-depth':6 'cu':4 ...,"{'brand': ['Frigidaire', 'Frigidaire'], 'model...",Ample Door Storage User Friendly Visibility,Feels solid and “upscale”. Excellent design of...,5.0,['1000289721'],1
2,bestbuy_us,Refrigerators,Whirlpool,Whirlpool - 21.9 Cu. Ft. Bottom-Freezer Refrig...,'21.9':2 'bottom':6 'bottom-freezer':5 'cu':3 ...,"{'brand': ['whirlpool'], 'Other_UPC': ['883049...",So much room,I love this fridge. So much room over having a...,5.0,"['3928039', '3928048', '3979801', '6112639']",4
3,bestbuy_us,Refrigerators,Whirlpool,Whirlpool - 21.9 Cu. Ft. Bottom-Freezer Refrig...,'21.9':2 'black':9 'bottom':6 'bottom-freezer'...,"{'brand': ['whirlpool'], 'Other_UPC': ['883049...",So much room,I love this fridge. So much room over having a...,5.0,"['3928039', '3928048', '3979801', '6112639']",4
4,bestbuy_us,Refrigerators,Whirlpool,Whirlpool - 22.1 Cu. Ft. Bottom-Freezer Refrig...,'22.1':2 'black':9 'bottom':6 'bottom-freezer'...,"{'brand': ['whirlpool'], 'Other_UPC': ['883049...",So much room,I love this fridge. So much room over having a...,5.0,"['3928039', '3928048', '3979801', '6112639']",4


## 1- Limpar e arrumar conjunto de texto

### 1.1 - Limpando Valores Nulos e Duplicados

In [4]:
data.fillna('', inplace=True)
data.isnull().sum()

Website               0
Product               0
Brand                 0
Description           0
Technical Features    0
General Features      0
Comment#1             0
Comment#2             0
Rating                0
Available Colors      0
No. Colors            0
dtype: int64

In [5]:
len(data[data.duplicated() == True])

518

In [6]:
data.drop_duplicates(inplace=True)
data.reset_index(inplace=True)
data.head()

,index,Website,Product,Brand,Description,Technical Features,General Features,Comment#1,Comment#2,Rating,Available Colors,No. Colors
0,0,lowes,Refrigerators,General Electric,GE 25.1-cu ft Side-by-Side Refrigerator with I...,'25.1':2 'black':13 'by':7 'cu':3 'ft':4 'ge':...,"{'brand': ['GE', 'Ge'], 'model': ['GSS25IBNTS'...",Functional,Pros: fingerprint resistant so you don't have ...,3.0,"['1000859768', '1000859852']",2
1,1,lowes,Refrigerators,Frigidaire,Frigidaire Gallery 21.7-cu ft Counter-depth Fr...,'21.7':3 'counter':7 'counter-depth':6 'cu':4 ...,"{'brand': ['Frigidaire', 'Frigidaire'], 'model...",Ample Door Storage User Friendly Visibility,Feels solid and “upscale”. Excellent design of...,5.0,['1000289721'],1
2,2,bestbuy_us,Refrigerators,Whirlpool,Whirlpool - 21.9 Cu. Ft. Bottom-Freezer Refrig...,'21.9':2 'bottom':6 'bottom-freezer':5 'cu':3 ...,"{'brand': ['whirlpool'], 'Other_UPC': ['883049...",So much room,I love this fridge. So much room over having a...,5.0,"['3928039', '3928048', '3979801', '6112639']",4
3,3,bestbuy_us,Refrigerators,Whirlpool,Whirlpool - 21.9 Cu. Ft. Bottom-Freezer Refrig...,'21.9':2 'black':9 'bottom':6 'bottom-freezer'...,"{'brand': ['whirlpool'], 'Other_UPC': ['883049...",So much room,I love this fridge. So much room over having a...,5.0,"['3928039', '3928048', '3979801', '6112639']",4
4,4,bestbuy_us,Refrigerators,Whirlpool,Whirlpool - 22.1 Cu. Ft. Bottom-Freezer Refrig...,'22.1':2 'black':9 'bottom':6 'bottom-freezer'...,"{'brand': ['whirlpool'], 'Other_UPC': ['883049...",So much room,I love this fridge. So much room over having a...,5.0,"['3928039', '3928048', '3979801', '6112639']",4


### 1.2 - Limpar o Texto. Pontuações, tudo em letra minúscula, stopwords, etc.

In [7]:
def remove_stop_words(text, stopwords):
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word for word in text_tokens if not word in stopwords]
    text = ' '.join(tokens_without_sw)
    return text

In [8]:
def clean_text(text, stopwords):
    text = text.lower()
    
    # Remove palavras entre colchetes
    text = re.sub('\[.*?\]', '', text)
    
    #remove palavras com numero
    text = re.sub('\w*\d\w*', '', text)
    
    #remove pontuação
    punctuation = '[' + re.escape(string.punctuation) +'\\”\\“'+']'
    text = re.sub(punctuation, '', text)
    
    #remove stop words
    text = remove_stop_words(text, stopwords)
    return text

In [9]:
col_name = ['Comment#1', 'Comment#2']
en_stopwords = stopwords.words('english')

new_words = ['aahs', 'would', 'etc','family', 'yrs', 'yoy', 'every','time', 'ive', 'youve', 'theres', 'youre', 'son', 'young', 'youll', 'aboveand', 'yin', 'yo', 'youl']
for word in new_words:
    en_stopwords.append(word)
data['Comment#1'] = data.apply(lambda x: clean_text(x[col_name[0]], en_stopwords), axis=1 )
data['Comment#2'] = data.apply(lambda x: clean_text(x[col_name[1]], en_stopwords), axis=1 )
data['Comment#2'][1]

'feels solid upscale excellent design door shelves allow depth easy visibility ditto storage drawers freezer love dual ice makers look enjoy fridge long'

### 1.3 - Juntar os dois tipos de comentário em um só

In [10]:
col_names = ['Comment#1', 'Comment#2']
data['Full_Comment'] = data.apply(lambda x: x[col_names[0]] + ' ' + x[col_names[1]], axis=1)
data.head(5)

,index,Website,Product,Brand,Description,Technical Features,General Features,Comment#1,Comment#2,Rating,Available Colors,No. Colors,Full_Comment
0,0,lowes,Refrigerators,General Electric,GE 25.1-cu ft Side-by-Side Refrigerator with I...,'25.1':2 'black':13 'by':7 'cu':3 'ft':4 'ge':...,"{'brand': ['GE', 'Ge'], 'model': ['GSS25IBNTS'...",functional,pros fingerprint resistant dont constantly wip...,3.0,"['1000859768', '1000859852']",2,functional pros fingerprint resistant dont con...
1,1,lowes,Refrigerators,Frigidaire,Frigidaire Gallery 21.7-cu ft Counter-depth Fr...,'21.7':3 'counter':7 'counter-depth':6 'cu':4 ...,"{'brand': ['Frigidaire', 'Frigidaire'], 'model...",ample door storage user friendly visibility,feels solid upscale excellent design door shel...,5.0,['1000289721'],1,ample door storage user friendly visibility fe...
2,2,bestbuy_us,Refrigerators,Whirlpool,Whirlpool - 21.9 Cu. Ft. Bottom-Freezer Refrig...,'21.9':2 'bottom':6 'bottom-freezer':5 'cu':3 ...,"{'brand': ['whirlpool'], 'Other_UPC': ['883049...",much room,love fridge much room french door style large ...,5.0,"['3928039', '3928048', '3979801', '6112639']",4,much room love fridge much room french door st...
3,3,bestbuy_us,Refrigerators,Whirlpool,Whirlpool - 21.9 Cu. Ft. Bottom-Freezer Refrig...,'21.9':2 'black':9 'bottom':6 'bottom-freezer'...,"{'brand': ['whirlpool'], 'Other_UPC': ['883049...",much room,love fridge much room french door style large ...,5.0,"['3928039', '3928048', '3979801', '6112639']",4,much room love fridge much room french door st...
4,4,bestbuy_us,Refrigerators,Whirlpool,Whirlpool - 22.1 Cu. Ft. Bottom-Freezer Refrig...,'22.1':2 'black':9 'bottom':6 'bottom-freezer'...,"{'brand': ['whirlpool'], 'Other_UPC': ['883049...",much room,love fridge much room french door style large ...,5.0,"['3928039', '3928048', '3979801', '6112639']",4,much room love fridge much room french door st...


## 2 - Avaliando Sentimento usando TextBlob Sentiment Package

In [11]:
col_name = ['Full_Comment']

data['polarity'] = data.apply(lambda x: TextBlob(x[col_name[0]]).sentiment.polarity, axis=1 )
data['subjectivity'] = data.apply(lambda x: TextBlob(x[col_name[0]]).sentiment.subjectivity, axis=1 )
data.head(5)

,index,Website,Product,Brand,Description,Technical Features,General Features,Comment#1,Comment#2,Rating,Available Colors,No. Colors,Full_Comment,polarity,subjectivity
0,0,lowes,Refrigerators,General Electric,GE 25.1-cu ft Side-by-Side Refrigerator with I...,'25.1':2 'black':13 'by':7 'cu':3 'ft':4 'ge':...,"{'brand': ['GE', 'Ge'], 'model': ['GSS25IBNTS'...",functional,pros fingerprint resistant dont constantly wip...,3.0,"['1000859768', '1000859852']",2,functional pros fingerprint resistant dont con...,0.250000,0.645833
1,1,lowes,Refrigerators,Frigidaire,Frigidaire Gallery 21.7-cu ft Counter-depth Fr...,'21.7':3 'counter':7 'counter-depth':6 'cu':4 ...,"{'brand': ['Frigidaire', 'Frigidaire'], 'model...",ample door storage user friendly visibility,feels solid upscale excellent design door shel...,5.0,['1000289721'],1,ample door storage user friendly visibility fe...,0.379762,0.561905
2,2,bestbuy_us,Refrigerators,Whirlpool,Whirlpool - 21.9 Cu. Ft. Bottom-Freezer Refrig...,'21.9':2 'bottom':6 'bottom-freezer':5 'cu':3 ...,"{'brand': ['whirlpool'], 'Other_UPC': ['883049...",much room,love fridge much room french door style large ...,5.0,"['3928039', '3928048', '3979801', '6112639']",4,much room love fridge much room french door st...,0.219048,0.271429
3,3,bestbuy_us,Refrigerators,Whirlpool,Whirlpool - 21.9 Cu. Ft. Bottom-Freezer Refrig...,'21.9':2 'black':9 'bottom':6 'bottom-freezer'...,"{'brand': ['whirlpool'], 'Other_UPC': ['883049...",much room,love fridge much room french door style large ...,5.0,"['3928039', '3928048', '3979801', '6112639']",4,much room love fridge much room french door st...,0.219048,0.271429
4,4,bestbuy_us,Refrigerators,Whirlpool,Whirlpool - 22.1 Cu. Ft. Bottom-Freezer Refrig...,'22.1':2 'black':9 'bottom':6 'bottom-freezer'...,"{'brand': ['whirlpool'], 'Other_UPC': ['883049...",much room,love fridge much room french door style large ...,5.0,"['3928039', '3928048', '3979801', '6112639']",4,much room love fridge much room french door st...,0.219048,0.271429


In [12]:
data['polarity'].mean()

0.2884663571042482

### 2.1 - Avaliando Polaridade para cada Marca

In [13]:
num_products_per_brand = data.groupby(['Brand'])['Product'].count().sort_values(ascending=False)
ten_most_brands = num_products_per_brand.head(10)
ten_most_brands

Brand
Whirlpool           6152
General Electric    4886
Frigidaire          2890
Samsung             1854
LG                  1784
Kitchenaid           837
Maytag               436
Cafe                 303
Insignia™            161
Magic Chef           120
Name: Product, dtype: int64

In [14]:
brands_mean_polarity = data.groupby('Brand')['polarity'].mean().sort_values(ascending=False)
brands_mean_polarity.head(10)

Brand
Nostalgia          0.600000
Unique             0.445000
Farberware         0.429365
Thomson            0.380134
Edgestar           0.334821
Cosmo              0.328571
Kitchenaid         0.315462
Whirlpool          0.300577
Wine Enthusiast    0.292391
Frigidaire         0.292159
Name: polarity, dtype: float64

Da marca Nostalgia até a marca Cosmo, eu só tenho marcas que tem entre 1 e 3 review no dataset, então isso fica bem dificil de avaliar.

In [15]:
brands_mean_polarity.loc[ten_most_brands.index]

Brand
Whirlpool           0.300577
General Electric    0.286678
Frigidaire          0.292159
Samsung             0.289545
LG                  0.255941
Kitchenaid          0.315462
Maytag              0.263659
Cafe                0.269668
Insignia™           0.275634
Magic Chef          0.283532
Name: polarity, dtype: float64

### 2.2 - Avaliando Polaridade para cada Website

In [16]:
num_products_per_website = data.groupby(['Website'])['Product'].count().sort_values(ascending=False)
ten_most_websites = num_products_per_website.head(10)
ten_most_websites

Website
lowes              9478
homedepot          4041
bestbuy_us         2406
walmart_us          987
whirlpool           791
costco_us           738
ge_appliance_us     490
lg_us               453
samsclub            196
maytag              187
Name: Product, dtype: int64

In [17]:
sites_mean_polarity = data.groupby('Website')['polarity'].mean().sort_values(ascending=False)
sites_mean_polarity.head(10)

Website
bestbuy_us    0.304749
lowes         0.297306
samsung_us    0.297235
samsclub      0.288169
walmart_us    0.284427
target        0.282605
costco_us     0.278420
homedepot     0.275252
maytag        0.267472
whirlpool     0.264419
Name: polarity, dtype: float64

In [18]:
sites_mean_polarity.loc[ten_most_websites.index]

Website
lowes              0.297306
homedepot          0.275252
bestbuy_us         0.304749
walmart_us         0.284427
whirlpool          0.264419
costco_us          0.278420
ge_appliance_us    0.262334
lg_us              0.237985
samsclub           0.288169
maytag             0.267472
Name: polarity, dtype: float64

### 2.3 - Avaliando Polaridade para cada Modelo de geladeira

In [19]:
def get_features(general_features):

    features = {}
    count = 0

    x = general_features.split('], ')
    for string in x:
        y = string.strip('{}')
        if(y.find('"') == -1):
            y = y.split("': ")
        else:
            y = y.split('": ')
        #print(y)
        for text in y:
            if(count==0):
                count=1
                if(text.strip(" '") == 'title'):
                    count= 0 
                    continue
                if(text.find('[') == -1):
                    chave = text.strip(" '").lower()
                    features[chave]=None
            else:
                count=0
                if(text.find('[') == -1):
                    continue
                else:
                    value = text.strip('[ ').split(',')
                    features[chave] = value
    for chave, value in features.items():
        if(chave.find('unspsc') != -1):
            features['unspsc'] = features.pop(chave)
            break
    return features

In [20]:
temp = data.copy()
general_features = data['General Features']
count = 0
for row in general_features:
    features = get_features(row)
    temp.at[count, 'General Features'] =  features
    count+=1
temp['General Features'][0].keys()

dict_keys(['brand', 'model', 'style', 'built-in', 'warranty', 'ice maker', 'sub-brand', 'depth type', 'door alarm', 'door style', 'frost-free', 'breadcrumb', 'handle type', 'series name', 'door shelves', 'door texture', 'handle color', 'hidden hinge', 'hub required', 'ada compliant', 'cabinet color', 'freezer light', 'see-thru door', 'shelf options', 'air filtration', 'safety listing', 'works with ios', 'dispenser light', 'freezer baskets', 'freezer shelves', 'door within door', '"lowe\'s exclusive', 'toe grille color', 'voice controlled', 'water filtration', 'voice_controlled', 'app compatibility', 'dispenser options', 'freezer door bins', 'refrigerator size', 'snack/deli drawer', 'cee tier qualified', 'child safety locks', 'works with android', 'gallon door storage', 'wi-fi compatibility', 'freezer door shelves', 'actual width (inches)', 'common width (inches)', 'energy star certified', 'fingerprint-resistant', 'appliance color/finish', 'water filter indicator', 'works with smartthin

In [21]:
col_name = ['General Features']
temp['Model'] = temp.apply(lambda x: 'model' in x[col_name[0]].keys(), axis=1)

In [22]:
data_w_model = temp.loc[temp['Model'] == True]

In [23]:
data_w_model['Model'] = data_w_model.apply(lambda x: x[col_name[0]]['model'][0], axis =1)

/home/joaogarcia/Documents/DataScience/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
num_products_per_model = data_w_model.groupby(['Model'])['Product'].count().sort_values(ascending=False)
ten_most_models = num_products_per_model.head(10)
ten_most_models

Model
'WRX735SDHZ'    141
'FFHB2750TD'    116
'GFE26JBMTS'    110
'FFSS2615TD'    105
'FFSS2615TS'    104
'FFSS2615TP'    103
'WRF535SWHZ'     95
'WRX735SDHW'     88
'FFTR1814TW'     87
'WRX735SDHV'     87
Name: Product, dtype: int64

In [25]:
models_mean_polarity = data_w_model.groupby('Model')['polarity'].mean().sort_values(ascending=False)
models_mean_polarity.head(10)

Model
'LUPXS3186N'        0.825000
'RF23M8070SG'       0.766667
'RF23M8070SG_AA'    0.766667
'RF23M8070SR'       0.766667
'RF23M8090SG'       0.766667
'RF23M8070DT_AA'    0.766667
'RF23M8090SR'       0.766667
'WRS973CIHZ'        0.675000
'WRS973CIHV'        0.675000
'KRBR102ESS'        0.650000
Name: polarity, dtype: float64

In [26]:
models_mean_polarity.loc[ten_most_models.index].sort_values(ascending=False)

Model
'WRF535SWHZ'    0.343611
'WRX735SDHW'    0.340473
'WRX735SDHV'    0.338783
'FFHB2750TD'    0.337564
'WRX735SDHZ'    0.310292
'FFSS2615TD'    0.298022
'FFSS2615TP'    0.296507
'FFSS2615TS'    0.296080
'FFTR1814TW'    0.287678
'GFE26JBMTS'    0.264681
Name: polarity, dtype: float64

In [27]:
models_mean_rating = data_w_model.groupby('Model')['Rating'].mean().sort_values(ascending=False)
models_mean_rating.head(10)

Model
'LSXC22486S'         5.0
'GCE06GGHBB'         5.0
'RS22T5561SR']       5.0
'RS22T5201SR'        5.0
'RS22T5201SG'        5.0
'RS22HDHPNSR'        5.0
'GWE23GENDS'         5.0
'GYE22GENDS'         5.0
'RF28R7201SR_AA'     5.0
'RF28R7201SR/AA']    5.0
Name: Rating, dtype: float64

In [28]:
models_mean_rating.loc[ten_most_models.index].sort_values(ascending=False)

Model
'WRX735SDHW'    4.534091
'WRX735SDHV'    4.528736
'WRF535SWHZ'    4.526316
'FFHB2750TD'    4.379310
'WRX735SDHZ'    4.333333
'FFTR1814TW'    4.321839
'GFE26JBMTS'    4.163636
'FFSS2615TP'    4.126214
'FFSS2615TD'    4.123810
'FFSS2615TS'    4.115385
Name: Rating, dtype: float64

### 2.4 - Avaliando o Modelo WRX735SDHZ - que tem a terceira melhor média de rating, e a primeira em polaridade, e é o modelo com mais comentários no dataset

In [29]:
WRX735SDHZ_info = data_w_model.loc[data_w_model['Model'] == "'WRX735SDHZ'" ]
WRX735SDHZ_info.reset_index(inplace=True)
len(WRX735SDHZ_info)

141

In [30]:
WRX735SDHZ_info['Website'].unique()

array(['costco_us', 'lowes'], dtype=object)

In [31]:
WRX735SDHZ_info['Brand'].unique()

array(['Whirlpool'], dtype=object)

In [32]:
WRX735SDHZ_info['Description'].unique()

array(['Whirlpool 25CuFt 36-inch Wide French Door Refrigerator with Accu-Chill Management System, Fingerprint Resistant Stainless Steel',
       'Whirlpool 24.5-cu ft 4-Door French Door Refrigerator with Ice Maker (Fingerprint-Resistant Stainless Steel) ENERGY STAR'],
      dtype=object)

In [33]:
WRX735SDHZ_info['Technical Features'][0]

"'25cuft':2 '36':3 'accu':11 'accu-chill':10 'chill':12 'door':7 'fingerprint':15 'french':6 'inch':4 'management':13 'refrigerator':8 'resistant':16 'stainless':17 'steel':18 'system':14 'whirlpool':1 'wide':5 'with':9"

In [34]:
WRX735SDHZ_info['General Features'][0]

{'brand': ["'whirlpool'"],
 'color': ["'Stainless Steel'"],
 'model': ["'WRX735SDHZ'"],
 'weight': ["'336 lb.'"],
 'features': ["'Water Filter'"],
 'fit width': ["'34 - 36 in.'"],
 'model number': ["'WRX735SDHZ'"],
 'model_number': ["'WRX735SDHZ'"],
 'warranty type': ["'Appliances'"],
 'freezer capacity': ["'6.73 cu. ft.'"],
 'freezer features': ["'Interior LED Lighting'"],
 'overall capacity': ["'20 - 24.9 cu. ft.'"],
 'refrigerator type': ["'French Door'"],
 'dimensions (wxdxh)': ["'36 in. x 35 in. x 70 in.'"],
 'refrigerator capacity': ["'17.78 cu. ft.'"],
 'refrigerator features': ["'Water Filter'"],
 'refrigerator shelf material': ["'Glass'"],
 'dimensions - w/ door open 90° (wxdxh)': ["'41 in. x 48 in. x 70.13 in.'"],
 'dimensions - w/o hinges & door (wxdxh)': ["'35.63 in. x 29 in. x 68.63 in.'"],
 'dimensions - w/ door & w/o handles (wxdxh)': ["'70.13 in. x 32.13 in. x 35.63 in.'"],
 'dimensions - w/ hinges, handles & door (wxdxh)': ["'35.63 in. x 34.63 in. x 70.13 in.']"]}

## 3 - Extraindo Features que andam falando sobre esse modelo

In [35]:
count_vector = CountVectorizer(stop_words=en_stopwords, ngram_range=(2,2))
words_count_data = count_vector.fit_transform(WRX735SDHZ_info.Full_Comment)
dtm = pd.DataFrame(words_count_data.toarray(), columns=count_vector.get_feature_names())
dtm.index = WRX735SDHZ_info.index
dtm.head()

,able get,able organize,absolute piece,absolutely love,absolutely obnoxious,acceptable household,access frequently,access negative,access visibility,accessibility wont,...,years ago,years american,years disappointing,years far,years non,years two,yes noise,yet checked,yet doors,yogurt thing
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
dtm.sum().sort_values(ascending=False).head(50)

ice maker              31
extra drawer           13
side side              11
love extra             11
french door            10
left door              10
great features         10
bought fridge          10
middle drawer           9
weeks ago               9
really like             8
month ago               8
great fridge            7
fridge month            7
lot space               6
drawer great            6
features bought         6
whirlpool fridge        6
love fridge             6
freezer drawer          6
much room               6
bought weeks            6
fridge ever             6
deli drawer             6
old fridge              6
lots space              6
love drawer             6
water ice               6
side door               5
right side              5
ago love                5
fridge bought           5
works great             5
room love               5
drawer perfect          5
pull drawer             5
plenty room             5
door doesnt             5
new fridge  

In [37]:
new_words = ['lol', 'love', 'bought', 'great', 'im', 'fridge', 'like', 'lot', 'happy', 'good', 'sure', 'best']
for word in new_words:
    en_stopwords.append(word)

In [38]:
count_vector = CountVectorizer(stop_words=en_stopwords, ngram_range=(2,2))
words_count_data = count_vector.fit_transform(WRX735SDHZ_info.Full_Comment)
dtm = pd.DataFrame(words_count_data.toarray(), columns=count_vector.get_feature_names())
dtm.index = WRX735SDHZ_info.index
dtm.head()

,able get,able organize,absolute piece,absolutely first,absolutely new,absolutely obnoxious,acceptable household,access frequently,access negative,access visibility,...,years ago,years american,years disappointing,years far,years non,years two,yes noise,yet checked,yet doors,yogurt thing
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
dtm.sum().sort_values(ascending=False).head(30)

ice maker             31
extra drawer          13
side side             11
french door           10
left door             10
middle drawer          9
weeks ago              9
freezer drawer         8
month ago              8
much room              6
water ice              6
lots space             6
deli drawer            6
refrigerator month     6
pull drawer            5
door doesnt            5
drawer perfect         5
plenty room            5
right side             5
side door              5
second drawer          4
water dispenser        4
refrigerator one       4
bit noisy              4
new one                4
getting used           4
nothing problems       4
still getting          4
drawer warped          4
drawer really          4
dtype: int64

### INSIGHTS:
A galera falou bastante Ice Maker, e do drawer. Falaram bastante da porta também. 

## 4 - Extraindo Features baseado em Marcas/Websites

### 4.1 - Whirlpool

In [40]:
whirlpool_data = data.loc[data['Brand'] == 'Whirlpool']
whirlpool_data.head()

,index,Website,Product,Brand,Description,Technical Features,General Features,Comment#1,Comment#2,Rating,Available Colors,No. Colors,Full_Comment,polarity,subjectivity
2,2,bestbuy_us,Refrigerators,Whirlpool,Whirlpool - 21.9 Cu. Ft. Bottom-Freezer Refrig...,'21.9':2 'bottom':6 'bottom-freezer':5 'cu':3 ...,"{'brand': ['whirlpool'], 'Other_UPC': ['883049...",much room,love fridge much room french door style large ...,5.0,"['3928039', '3928048', '3979801', '6112639']",4,much room love fridge much room french door st...,0.219048,0.271429
3,3,bestbuy_us,Refrigerators,Whirlpool,Whirlpool - 21.9 Cu. Ft. Bottom-Freezer Refrig...,'21.9':2 'black':9 'bottom':6 'bottom-freezer'...,"{'brand': ['whirlpool'], 'Other_UPC': ['883049...",much room,love fridge much room french door style large ...,5.0,"['3928039', '3928048', '3979801', '6112639']",4,much room love fridge much room french door st...,0.219048,0.271429
4,4,bestbuy_us,Refrigerators,Whirlpool,Whirlpool - 22.1 Cu. Ft. Bottom-Freezer Refrig...,'22.1':2 'black':9 'bottom':6 'bottom-freezer'...,"{'brand': ['whirlpool'], 'Other_UPC': ['883049...",much room,love fridge much room french door style large ...,5.0,"['3928039', '3928048', '3979801', '6112639']",4,much room love fridge much room french door st...,0.219048,0.271429
5,5,bestbuy_us,Refrigerators,Whirlpool,Whirlpool - 21.9 Cu. Ft. Bottom-Freezer Refrig...,'21.9':2 'bottom':6 'bottom-freezer':5 'cu':3 ...,"{'brand': ['whirlpool'], 'Other_UPC': ['883049...",much room,love fridge much room french door style large ...,5.0,"['3928039', '3928048', '3979801', '6112639']",4,much room love fridge much room french door st...,0.219048,0.271429
9,9,costco_us,Refrigerators,Whirlpool,Whirlpool 28CuFt Side-by-Side Refrigerator in ...,'28cuft':2 'and':16 'by':5 'dispenser':18 'eve...,"{'Brand': ['Whirlpool'], 'Color': ['Stainless ...",good product,bought product office pretty good exception wa...,4.0,['100369172'],1,good product bought product office pretty good...,0.550000,0.733333


In [41]:
count_vector = CountVectorizer(stop_words=en_stopwords, ngram_range=(2,2))
words_count_data = count_vector.fit_transform(whirlpool_data.Full_Comment)
dtm = pd.DataFrame(words_count_data.toarray(), columns=count_vector.get_feature_names())
dtm.index = whirlpool_data.index
dtm.head()

,ability change,ability ice,ability store,able big,able change,able compartments,able delivered,able find,able fit,able fix,...,yogurt thing,youtube videos,yr old,zero cant,zero degrees,zero freezer,zone freezer,zone manual,zone two,zones work
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
dtm.sum().sort_values(ascending=False).head(30)

ice maker                    1138
side side                     511
weeks ago                     358
month ago                     356
water dispenser               302
french door                   270
lots room                     255
ice water                     242
stainless steel               223
old one                       204
water ice                     201
plenty room                   191
new refrigerator              180
freezer bottom                166
bottom freezer                160
year old                      152
much room                     151
works well                    148
months ago                    140
freezer door                  137
freezer space                 134
refrigerator refrigerator     133
lots space                    129
easy access                   120
door refrigerator             117
door shelves                  115
old refrigerator              114
whirlpool refrigerator        112
new one                       112
top freezer   

### Tentativa 1: Retirar dos texto palavras mais subjetivas, usando o paramêtro do TextBlob

In [43]:
for col in dtm.columns:
    tokens = word_tokenize(col)
    for token in tokens:
        if((TextBlob(token).sentiment.subjectivity) > 0.2):
            en_stopwords.append(token)

In [44]:
count_vector = CountVectorizer(stop_words=en_stopwords, ngram_range=(2,2))
words_count_data = count_vector.fit_transform(whirlpool_data.Full_Comment)
dtm = pd.DataFrame(words_count_data.toarray(), columns=count_vector.get_feature_names())
dtm.index = whirlpool_data.index
dtm.head()

,ability change,ability ice,ability store,accept delivery,accept take,acceptable household,acceptable limits,acceptable replacement,acceptable size,acceptable suit,...,yogurt thing,youtube videos,yr old,zero cant,zero degrees,zero freezer,zone freezer,zone manual,zone two,zones work
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
dtm.sum().sort_values(ascending=False).head(30)

ice maker                    1138
side side                     513
weeks ago                     358
month ago                     356
water dispenser               302
french door                   270
ice water                     261
lots room                     255
stainless steel               223
old one                       216
water ice                     203
plenty room                   191
freezer bottom                166
bottom freezer                160
year old                      152
much room                     151
works well                    148
freezer door                  142
months ago                    140
refrigerator refrigerator     140
freezer space                 138
lots space                    129
would recommend               125
door refrigerator             117
old refrigerator              117
whirlpool refrigerator        116
door shelves                  115
room freezer                  114
refrigerator freezer          110
refrigerator m

## 4.2 Quais caraterísticas aparecem em ratings mais baixos?

In [60]:
whirlpool_data_bad = whirlpool_data.loc[whirlpool_data['Rating'] <=2]
len(whirlpool_data_bad)

465

In [61]:
count_vector = CountVectorizer(stop_words=en_stopwords, ngram_range=(2,2))
words_count_data = count_vector.fit_transform(whirlpool_data_bad.Full_Comment)
dtm = pd.DataFrame(words_count_data.toarray(), columns=count_vector.get_feature_names())
dtm.index = whirlpool_data_bad.index
dtm.head()

,accept take,acceptable limits,access compartment,access fraction,access handle,access made,accessories shelves,accidently cleaning,accidently probable,accommodate also,...,yes made,yesterday compressor,yet bottom,yet checked,youtube videos,yr old,zero cant,zone freezer,zone manual,zone two
109,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
129,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
130,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
131,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
205,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
new_words = ['every','time']
for word in new_words

SyntaxError: invalid syntax (<ipython-input-48-8321415ca40a>, line 2)

In [63]:
thirty_most_bad = (dtm.sum()).sort_values(ascending=False).head(30)
thirty_most_bad/(465)

ice maker                 0.268817
freezer door              0.077419
stopped working           0.060215
shelves bins              0.058065
customer service          0.043011
less volume               0.038710
internal shelves          0.038710
inside door               0.036559
door shelves              0.034409
design flaw               0.034409
water dispenser           0.034409
years ago                 0.032258
maker stopped             0.030108
door handles              0.030108
weeks ago                 0.030108
home depot                0.030108
two years                 0.030108
water line                0.030108
food frozen               0.027957
refrigerator delivered    0.027957
whirlpool product         0.027957
door takes                0.025806
freezer keeps             0.025806
year ago                  0.025806
didnt work                0.025806
worth paid                0.025806
old samsung               0.025806
month ago                 0.025806
nothing problems    

## Insights

Olhando essa palavras, e a porcentagem com que elas aparecem:
- A palavra **ice maker** apareceu em 27% dos comentários
- Temos algumas palavras relacionadas ao fato de que algo parou de funcionar
   - **didnt work** apareceu em 2.5%, **stopped working** (6%), **maker stopped**(3%)
- Temos reclamações com relação as portas:
    - a palavra **door** apareceu várias vezes como em **freezer door**(7%), **inside door**(3,65%), **door handles** (3%), **door shelves** (3,45%)
- Temos palavras relacionadas a tempo:
    - como **month ago**, **year ago**, **years ago**, **two years**, **weeks ago**

#### A partir disso podemos supor algumas coisas
- Os usuários tenham problemas com a geladeira, depois de pelo menos algumas semanas utilizando, alguns até anos depois
- A maioria dos problemas relatados são com as portas da geladeira e do freezer
- Mas há também vários problemas com o atendimento ao cliente (customer service(4,3%)), e as prateleiras(shelves bins (5,8%) e door shelves (3,45%)
- Além disso, aquelas torneiras para por água e para por gelo também são motivo de reclamação (ice dispenser (2.3%)) e (water dispenser (3,45%)

## 4.3 E Mais positivos?

In [74]:
whirlpool_data_good = whirlpool_data.loc[whirlpool_data['Rating'] > 3]
len(whirlpool_data_good)

5116

In [75]:
count_vector = CountVectorizer(stop_words=en_stopwords, ngram_range=(2,2))
words_count_data = count_vector.fit_transform(whirlpool_data_good.Full_Comment)
dtm = pd.DataFrame(words_count_data.toarray(), columns=count_vector.get_feature_names())
dtm.index = whirlpool_data_good.index
dtm.head()

,ability change,ability ice,ability store,acceptable household,acceptable size,acceptable suit,access access,access anything,access area,access bottom,...,yet request,yet room,yet still,yet water,yet wider,yogurt thing,yr old,zero degrees,zero freezer,zones work
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1]:
thirty_most_good = (dtm.sum()).sort_values(ascending=False).head(30)
thirty_most_good/len(whirlpool_data_good)

NameError: name 'dtm' is not defined

## Insights:

- A palavra **Ice Maker** aparece 16% dos reviews, o que é menos vezes que nos comentários negativos.
    - Duas respostas para isso: ou é uma caraterística tão comum que a galera esquece de elogiar
                                ou ela quebra mais que funciona de verdade
                               